# 유사도 계산으로 판례댓글과 유사한 일반댓글 필터링

In [108]:
import pandas as pd
import numpy as np

In [109]:
dat = pd.read_csv('C:/Users/mrg07/Downloads/유사도계산.csv')


KeyboardInterrupt



In [172]:
df_origin_innocent = pd.read_csv('C:/Users/mrg07/Desktop/innocent_유머.csv', encoding = 'cp949')
df_origin_innocent.columns = ['comments', 'year']

In [117]:
dat["label0"] = dat["label0"].astype("str")
dat["label1"] = dat["label1"].astype("str")
df_origin_innocent["comments"] = df_origin_innocent["comments"].astype("str")

# 왜인지 label 열에 숫자형 데이터가 남아있어서 한번 싹 형변환!

In [112]:
len(df_origin_innocent)

475097

In [118]:
dat['length'] = dat['label0'].apply(len)
print(dat['length'].max())
dat = dat[dat['length'] <= 300]
print(dat['length'].max())
dat = dat.drop('length', axis = 1)

4127
295


In [119]:
df_origin_innocent['length'] = df_origin_innocent['comments'].apply(len)
print(df_origin_innocent['length'].max())
df_origin_innocent = df_origin_innocent[df_origin_innocent['length'] <= 300]
print(df_origin_innocent['length'].max())
df_origin_innocent = df_origin_innocent.drop('length', axis = 1)

300
300


In [120]:
import seaborn as sns
# sns.histplot(dat['score'], color='brown')

## 전처리

In [121]:
# label=0 (비속어) 지우기

import re

def remove_(sentence):
    pattern = r'\([^)]*\)'
    filtered_sentence = re.sub(pattern=pattern, repl="", string=sentence)
    
    return filtered_sentence

dat['label0']=dat['label0'].apply(remove_)
df_origin_innocent['comments']=df_origin_innocent['comments'].apply(remove_)

In [129]:
dat["label0"] = dat["label0"].apply(lambda x : ' '.join(x.split()))
df_origin_innocent["comments"] = df_origin_innocent["comments"].apply(lambda x : ' '.join(x.split()))

# 다중 공백 제거

In [131]:
len(dat)

25571340

## 유사도 결과

In [133]:
def get_top5_mean(x):
     return x.sort_values('score', ascending=False)[["score"]][:5].mean()


a=dat.groupby("label0").apply(get_top5_mean)
a = a.reset_index()
a = a.sort_values('score', ascending=False).reset_index(drop=True)

In [134]:
pd.set_option('display.max_colwidth', None)

In [135]:
innocent = a.loc[:854]
#n은 일반 판례 개수

In [136]:
innocent.head()

,label0,score
0,야이 개때끼야,0.763758
1,진짜 개같에,0.700923
2,개색,0.687995
3,뭐여이게 개 새 키 뭐얔,0.665487
4,현석 어떻게된거야,0.653540


In [137]:
import seaborn as sns
# sns.histplot(innocent['score'], color='brown')

In [138]:
innocent.drop('score', axis = 1, inplace = True)
innocent.columns = ['comments']

C:\Users\mrg07\AppData\Local\Temp\ipykernel_7504\313803156.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  innocent.drop('score', axis = 1, inplace = True)


### 일반댓글 길이와 연도 파생변수 붙이기

In [140]:
# 길이 붙이기
#innocent['comments'] = innocent['comments'].astype(str)
innocent['length'] = innocent['comments'].apply(len)

C:\Users\mrg07\AppData\Local\Temp\ipykernel_7504\4245080826.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  innocent['length'] = innocent['comments'].apply(len)


In [144]:
df_origin_innocent = df_origin_innocent.drop_duplicates('comments').reset_index(drop=True)
len(df_origin_innocent)

449833

In [145]:
df_origin_innocent.isna().sum()

comments    0
year        0
dtype: int64

In [146]:
# 연도 앞 4개만 가져옴
df_origin_innocent['year'] = df_origin_innocent['year'].str[0:4].astype(int)

In [147]:
df_origin_innocent

,comments,year
0,보물섬분들도 많이 받으세여어어어어어잇...,2021
1,팝콘 사세요,2021
2,메리 크리스마스.,2021
3,해피 할로윈,2021
4,세복많이 받으세용,2021
...,...,...
449828,와 대박 역시 폭소바겐,2020
449829,여태까지 본것중에 이게 제일 잘생김,2020
449830,"와, 공감",2020
449831,티나도 끝까지 보게되는 개마성,2020


In [148]:
asdf = pd.merge(innocent, df_origin_innocent, how = 'left', on=['comments'])
asdf.head()
print(len(asdf))

855


In [150]:
asdf.isna().sum()

comments    0
length      0
year        2
dtype: int64

아직도 결측치 있어 ㅠㅠ

In [151]:
asdf[asdf.isna().any(axis=1)]

,comments,length,year
396,"안녕하세요 보섬대학교에 재학 중인 피해자입니다 지금 이 영상의 주인공인 보물섬들 선배들에게 학교 폭력을 당해왔습니다 야 이 개 시 치킨 안 사와 등 협박과 셔틀은 하루에 10번 씩 당해왔습니다 제가 저항하면 보물섬 선배들이 무리를 끌고 와서 집단 구타를 했습니다. 제가 제발 그만하달라고 외쳤는데 보물섬 선배들이 그치만 이렇게 하지 않으면 나에게 관심이 없는 걸, 그래서 전 손나나, 그럴리가없잖아 난 매일 매일 너에게 치킨을 먹일려고 너에게 관심을 얻으랴고 했다구, 그러곤 방관하시던 이 5252 썩 보가 좋은 관경이러군",292,NaN
406,"아니 진치충인 들 많네 뭐 신고고 나발이고 니들 빠다로 후려쳐버리고 싶은데 이고 뭐고간에 재미있었음 됬지 싸부리네 댓글 읽으면서 쪼겠다 그리고 장난이 도를 넘었다는 들아, 사람들이 생각하는 장난 의 기준은 다 달라 아니 뭔 도를 넘어 좋게좋게 넘어갔음 된거고 하튼 는거 보기 싫으니까 제발 니들생각은 니들 안에 쳐넣고 쌉쳐",180,NaN


In [ ]:
# 이건 그냥 직접 붙이자.......

### 판례댓글과 일반댓글 결합

In [153]:
df = pd.read_csv('C:/Users/mrg07/Desktop/유무죄전까지_1111.csv')

In [154]:
len(df)

855

In [155]:
df.drop('guilty', axis = 1, inplace = True)

In [156]:
df.columns

Index(['text', '선고년도', 'comments', 'length'], dtype='object')

In [157]:
df.columns = ['text', 'year', 'comments', 'length']

In [158]:
df_final_1111 = pd.concat([df, asdf]).reset_index(drop=True)

In [166]:
df_final_1111[df_final_1111['year'].isna()]

,text,year,comments,length
1251,NaN,NaN,"안녕하세요 보섬대학교에 재학 중인 피해자입니다 지금 이 영상의 주인공인 보물섬들 선배들에게 학교 폭력을 당해왔습니다 야 이 개 시 치킨 안 사와 등 협박과 셔틀은 하루에 10번 씩 당해왔습니다 제가 저항하면 보물섬 선배들이 무리를 끌고 와서 집단 구타를 했습니다. 제가 제발 그만하달라고 외쳤는데 보물섬 선배들이 그치만 이렇게 하지 않으면 나에게 관심이 없는 걸, 그래서 전 손나나, 그럴리가없잖아 난 매일 매일 너에게 치킨을 먹일려고 너에게 관심을 얻으랴고 했다구, 그러곤 방관하시던 이 5252 썩 보가 좋은 관경이러군",292
1261,NaN,NaN,"아니 진치충인 들 많네 뭐 신고고 나발이고 니들 빠다로 후려쳐버리고 싶은데 이고 뭐고간에 재미있었음 됬지 싸부리네 댓글 읽으면서 쪼겠다 그리고 장난이 도를 넘었다는 들아, 사람들이 생각하는 장난 의 기준은 다 달라 아니 뭔 도를 넘어 좋게좋게 넘어갔음 된거고 하튼 는거 보기 싫으니까 제발 니들생각은 니들 안에 쳐넣고 쌉쳐",180


In [181]:
df_final_1111.at[1251, 'year'] = 2019

In [182]:
df_final_1111.at[1261, 'year'] = 2018

In [183]:
df_final_1111[df_final_1111['year'].isna()]

,text,year,comments,length


드디어 결측 없다!!!!!!!!!

In [187]:
df_final_1111.isna().sum()

text        855
year          0
comments      0
length        0
dtype: int64

In [189]:
# df_final_1111.to_csv('C:/Users/mrg07/Desktop/real_year_length_1112.csv', index = False)